# Markov Snippets THUG vs SMC-THUG on 2D Ellipse Example

### Problem Statement

Classic 2D ellipse example, $f(x) = \log\mathcal{N}(x\mid 0, \Sigma)$ where $\Sigma = \text{diag}(0.1, 1)$. We focus on a uniform prior on an ellipse, and use a uniform kernel leading to filamentary distributions of the type
$$
\eta_{\epsilon}(x) \propto \mathbb{I}(\|f(x) - y\| \leq \epsilon)
$$

### Markov-Snippets THUG (version 1)

Version one corresponds to Algorithm 1 in the manuscript and, given $\epsilon s = [\epsilon_0, \epsilon_1, \ldots, \epsilon_P]$, the number of bounces $B$, the number of particles $N$, and the step size $\delta$, proceeds as follows. Below $\psi(z)$ corresponds to using the THUG integrator with parameters $B$ and $\delta$ using the gradient $\nabla f$ and outputting the trajectory $\psi(z_0) = [z_0, z_1, \ldots, z_B]$.

- **Initialization**
    - Sample $x_0$ on the ellipse $\mathcal{M}$ up to numerical accuracy using geometric arguments.
    - Use RWM with step-size $\delta$ to sample from $\eta_{\epsilon_0}$ starting from $x_0$. Use a burn-in (of $100$ in this case) and thinning (of $10$ in this case). This gives us samples $x_0^{(1:N)}$ from $\eta_{\epsilon_0}$.
    - Sample velocities $v_0^{(1:N)}\sim \mathcal{N}(0, I)$.
    - Form initial particles $z_0^{(i)} = (x_0^{(i)}, v_0^{(i)})$ for each $i=1, \ldots, N$.
- **Main Loop**:
    - For each iteration $n=1, \ldots, P$:
        - Apply $\psi$: $Z^{(i)}_{n-1} = \psi(z_{n-1}^{(i)})$ this has dimension $(N, B + 1, 4)$.
        - Compute weights (and normalize them)
            $$
            \bar{w}_{n, k}^{(i)} = \frac{\eta_{\epsilon_n}(\psi^k(z_{n-1}^{(i)}))}{\eta_{\epsilon_{n-1}}(z_{n-1}^{(i)})} = \mathbb{I}(\|f(x_{n-1, k}^{(i)}) - y\| \leq \epsilon_n)
            $$
        - Resample particles using (normalized) weights $\bar{w}_{n, k}$
        - Refresh velocities

### Markov-Snippets THUG (version 2)

This version is pretty much identical to Version 1. The only difference is that here we use $\tilde{\psi}(z)$ which again performs THUG with $B$ bounces with step size $\delta$, however it only outputs the beginning and end of the trajectory, i.e. $\tilde{\psi}(z_0) = [z_0, z_B]$. The initialization is the samem, so we skip to the main loop.

- **Main Loop**:
    - For each iteration $n=1, \ldots, P$:
        - Apply $\tilde{\psi}$: $Z_{n-1}^{(i)} = \tilde{\psi}(z_{n-1}^{(i)})$. This has dimension $(N, 2, 4)$.
        - Compute weights the same way as version 1.
        - Resample particles using the weights.
        - Refresh velocities.

### SMC-THUG

This is pretty much a standard SMC sampler targeting the sequence of filamentary distributions. The initialization is the same as for the two algorithms above.


- **Main Loop**:
    - For each iteration $n = 1, \ldots, P$:
        - Refresh Velocities: $v_{n-1}^{(1:N)}\sim \mathcal{N}(0, I)$ and re-form particles $z_{n-1}^{(i)} = (x_{n-1}^{(i)}, v_{n-1}^{(i)})$.
        - Run THUG sampler (not just the integrator, this has a Metropolis-Hastings step in it) with $B$ bounces and step size $\delta$ for each of the starting particles and targeting $\eta_{\epsilon_{n-1}}$, thus obtaining $z_{n}^{(i)} = \text{THUG}_{B, \delta}(z_{n-1}^{(i)})$. Notice that here we do not output the whole trajectory, but we either output the initial point or the final point of the trajectory based on the MH step.
        - Compute weights
            $$
            w_n^{(i)} \propto \frac{\eta_{\epsilon_n}(x_{n-1}^{(i)})\varpi(v_{n-1}^{(i)})}{\eta_{\epsilon_{n-1}}(x_{n-1}^{(i)})\varpi(v_{n-1}^{(i)})} \propto \frac{\mathbb{I}(\|f(x_{n-1}^{(i)}) - y\| \leq \epsilon_n)}{\mathbb{I}(\|f(x_{n-1}^{(i)}) - y\| \leq \epsilon_{n-1}} \propto \mathbb{I}(\|f(x_{n-1}^{(i)}) - y\| \leq \epsilon_n)
            $$
        - Resample particles using weights.

### Old comments

- Aim: first attempt at coding the Markov Snippets SMC sampler with THUG mutation kernel, as described by Algorithm 1 in Christophe's notes. 
- Application: Here we apply it to the ellipse problem.
- Important: here we focus on $\alpha = 0$

A simplified version of Algorithm 1 is basically this:

- Initialize particles $z_0^{(1:N)}\sim \mu_0$.
- For each iteration $n=1, \ldots, P$: 
    1. Construct trajectories $z_{n-1, k}^{(1:N)}$ 
    2. Resample trajectory points down to $N$ particles using weights
    $$
    \bar{w}_{n, k} = \frac{\mu_n(z_{n-1, k}^{(i)})}{\mu_n(z_{n-1}^{(i)})}
    $$
    3. Rejuvenate velocities for the $N$ particles

In [672]:
import numpy as np
from numpy import zeros, eye, array, diag, exp
from numpy.linalg import solve, norm
from numpy.random import choice
from scipy.stats import multivariate_normal as MVN
import math
import time

import matplotlib.pyplot as plt
from matplotlib import rc

from Manifolds.GeneralizedEllipse import GeneralizedEllipse
from utils import prep_contour
from RWM import RWM
from tangential_hug_functions import HugTangential

#### Settings for the Ellipse and Filamentary Distributions

In [673]:
μ  = zeros(2)
Σ  = diag(array([0.1, 1]))
level_set_value = -2.9513586307684885
ellipse = GeneralizedEllipse(μ, Σ, exp(level_set_value))
πellipse = MVN(μ, Σ)
f = πellipse.logpdf
grad_f = lambda ξ: -solve(Σ, ξ - μ)

def generate_ηϵ(ϵ):
    """Generates ηϵ with a uniform kernel."""
    def ηϵ(x):
        if np.linalg.norm(f(x) - level_set_value) <= ϵ:
            return 1.0
        else: 
            return 0.0
    return ηϵ

def generate_logηϵ(ϵ):
    """As above, this is for the uniform kernel but this computes the log density."""
    def logηϵ(ξ):
        with np.errstate(divide='ignore'):
            return np.log(float(norm(f(ξ) - level_set_value) <= ϵ) / ϵ)
    return logηϵ

In [674]:
def find_bounding_box_for_ellipse(n):
    """Computes lots of samples of the ellipse to try and find a bounding box for it."""
    overall_minimum = 0.0
    overall_maximum = 0.0
    
    for i in range(n):
        point = ellipse.sample(advanced=True)
        minimum, maximum = np.min(point), np.max(point)
        if minimum < overall_minimum:
            overall_minimum = minimum
        if maximum > overall_maximum:
            overall_maximum = maximum
    
    return overall_minimum, overall_maximum

#### Settings for the THUG kernel

In [675]:
B = 20
δ = 0.1

In [676]:
def THUGIntegratorUnivariate(z0, B, δ, grad):
    """THUG Integrator for the 2D example (ie using gradients, not jacobians)."""
    trajectory = zeros((B + 1, len(z0)))
    x0, v0 = z0[:len(z0)//2], z0[len(z0)//2:]
    x, v = x0, v0
    trajectory[0, :] = z0
    # Integrate
    for b in range(B):
        x = x + δ*v/2
        g = grad(x)
        ghat = g / norm(g)
        v = v - 2*ghat*(ghat@v)
        x = x + δ*v/2
        trajectory[b+1, :] = np.hstack((x, v))
    return trajectory

def generate_THUGIntegratorUnivariate(B, δ):
    """Returns a THUG integrator for a given B and δ."""
    grad = lambda ξ: -solve(Σ, ξ - μ) #ellipse.Q(ξ).T.flatten()
    integrator = lambda z: THUGIntegratorUnivariate(z, B, δ, grad)
    return integrator

###############################
#### this is for \tilde{ψ}.
###############################
def THUGIntegratorUnivariateOnlyEnd(z0, B, δ, grad):
    """Similar to THUG integrator but one step does B bounces."""
    trajectory = zeros((2, len(z0)))
    x0, v0 = z0[:len(z0)//2], z0[len(z0)//2:]
    x, v = x0, v0
    trajectory[0, :] = z0
    # Integrate
    for _ in range(B):
        x = x + δ*v/2
        g = grad(x)
        ghat = g / norm(g)
        v = v - 2*ghat*(ghat@v)
        x = x + δ*v/2
    trajectory[1, :] = np.hstack((x, v))
    return trajectory

def generate_THUGIntegratorUnivariateOnlyEnd(B, δ):
    """Returns a THUG integratorOnlyEnd for a given B and δ."""
    grad = lambda ξ: -solve(Σ, ξ - μ) #ellipse.Q(ξ).T.flatten()
    integrator = lambda z: THUGIntegratorUnivariateOnlyEnd(z, B, δ, grad)
    return integrator


#### Metropolis-Hastings version for SMC version
def THUG_MH(z0, B, δ, logpi):
    """Similar to THUGIntegratoUnivariateOnlyEnd but this uses a MH step."""
    grad = lambda ξ: -solve(Σ, ξ - μ)
    x0, v0 = z0[:len(z0)//2], z0[len(z0)//2:]
    x, v = x0, v0
    logu = np.log(np.random.rand())
    for _ in range(B):
        x = x + δ*v/2
        g = grad(x)
        ghat = g / norm(g)
        v = v - 2*ghat*(ghat@v)
        x = x + δ*v/2
    if logu <= logpi(x) - logpi(x0):
        # accept new point
        return np.concatenate((x, v))
    else:
        # accept old point
        return z0

# Markov Snippets THUG (version 1)

In [677]:
class MarkovSnippetsTHUG:
    
    def __init__(self, N, B, δ, d, ϵs, onlyend=False):
        """Markov Snippets SMC samplers corresponding exactly to Algorithm 1 in Christophe's notes.
        It uses the THUG kernel as its mutation kernel. The sequence of distributions is fixed here 
        since we provide ϵs, i.e. a list of tolerances which automatically fully specify the posterior 
        distributions used at each round.
        
        Parameters
        ----------
        
        :param N: Number of particles
        :type N:  int
        
        :param B: Number of bounces for the THUG integrator. Equivalent to `L` Leapfrog steps in HMC.
        :type B: int
        
        :param δ: Step-size used at each bounce, for the THUG integrator.
        :type δ: float
        
        :param d: Dimensionality of the `x` component of each particle, and equally dimensionality of 
                  `v` component of each particle. Therefore each particle has dimension `2d`.
        
        :param ϵs: Tolerances that fully specify the sequence of target filamentary distributions.
        :type ϵs: iterable
        
        :param onlyend: If False, we just run MS-SMC with the THUG integrator as kernel. If True, the THUG 
                        integrator is pretty much the same, but it only outputs the initial and final point, 
                        as part of the trajectory. Inside the integrator, it still performs B bounces, but 
                        basically we are changing \psi to \tilde{\psi} which performs B steps at once, and we 
                        run it only once.
        :type onlyend: bool
        """
        # Input variables
        self.N  = N       
        self.B  = B
        self.δ  = δ
        self.d  = d
        self.ϵs = ϵs       
        
        # Variables derived from the above
        self.P  = len(ϵs) - 1                                       # Number of target distributions
        self.ϖ  = MVN(zeros(d), eye(d))                             # Distribution of the velocities
        self.ηs = [generate_ηϵ(ϵ) for ϵ in ϵs]                      # List of filamentary distributions 
#         self.μs = [lambda z: ηϵ(z[:self.d]) for ηϵ in self.ηs]
        if not onlyend:
            self.ψ = generate_THUGIntegratorUnivariate(B, δ)
        else:
            self.ψ = generate_THUGIntegratorUnivariateOnlyEnd(B, δ)
            self.B = 1
    
    def initialize_particles(self):
        """To initialize particles, we sample from a uniform on a large rectangle.
        A rectangle of size [-100, 100] should be plenty large."""
        # Initialize first position on the manifold
        x0 = ellipse.sample(advanced=True)
        # Generate log-density for self.η[0]
        logηϵ0 = generate_logηϵ(self.ϵs[0])
        # Sample using RWM
        burn_in = 100
        TO_BE_THINNED, _ = RWM(x0, self.δ, burn_in + 10*self.N, logηϵ0)
        # Thin the samples to obtain the particles
        initialized_particles = TO_BE_THINNED[burn_in:][::10]
        # Refresh velocities and form particles
        v0 = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
        z0 = np.hstack((initialized_particles, v0))
        self.starting_particles = z0
        return z0
        
    def sample(self):
        """Starts the Markov Snippets sampler."""
        starting_time = time.time() 
        N = self.N
        B = self.B
        ## Storage
        #### Store z_n^{(i)}
        self.ZN  = np.zeros((self.P+1, N, 2*self.d))
        #### Store z_{n, k}^{(i)} so basically all the N(T+1) particles
        self.ZNK  = np.zeros((self.P, N*(B+1), 2*self.d))
        self.Wbar = np.zeros((self.P, N*(B+1)))
        self.ESS  = np.zeros((self.P))
        # Initialize particles
        z = self.initialize_particles()   # (N, 2d)
        self.ZN[0] = z
        # For each target distribution, run the following loop
        for n in range(1, self.P+1):
            # Compute trajectories
            Z = np.apply_along_axis(self.ψ, 1, z) # should have shape (N, B+1, 2d)
            self.ZNK[n-1] = Z.reshape(N*(B+1), 2*self.d)
            # Compute weights.
            #### Denominator: shared for each point in the same trajectory
            μnm1_z  = np.apply_along_axis(self.ηs[n-1], 1, Z[:, 0, :self.d])              # (N, )
            μnm1_z  = np.repeat(μnm1_z, self.B+1, axis=0).reshape(N, B+1) # (N, B+1)
            #### Numerator: different for each point on a trajectory.
            μn_ψk_z = np.apply_along_axis(self.ηs[n], 2, Z[:, :, :self.d])                         # (N, B+1)
            #### Put weights together
            W = μn_ψk_z / μnm1_z #np.exp(log_μn_ψk_z - log_μnm1_z)
            #### Normalize weights
            W = W / W.sum()
            # store weights (remember these are \bar{w})
            self.Wbar[n-1] = W.flatten()
            # compute ESS
            self.ESS[n-1] = 1 / np.sum(W**2)
            # Resample down to N particles
            resampling_indeces = choice(a=np.arange(N*(B+1)), size=N, p=W.flatten())
            indeces = np.dstack(np.unravel_index(resampling_indeces, (N, B+1))).squeeze()
            z = np.vstack([Z[tuple(ix)] for ix in indeces])     # (N, 2d)
            
            # Rejuvenate velocities of N particles
            z[:, self.d:] = np.random.normal(loc=0.0, scale=1.0, size=(N, self.d))
            self.ZN[n] = z
        self.total_time = time.time() - starting_time
        return z

In [697]:
# At this point, doesn't really matter which ϵs we use.
ϵs = np.linspace(start=10, stop=0.001, num=20)
B = 50
δ = 0.1
N  = 5000
# Instantitate the algorithm
MSTHUG = MarkovSnippetsTHUG(N=N, B=B, δ=δ, d=2, ϵs=ϵs)

In [ ]:
# Sample
zP = MSTHUG.sample()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
for i in range(4):
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value])
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value-MSTHUG.ϵs[i], level_set_value+MSTHUG.ϵs[i]], colors='gray')
    ax[i].scatter(*MSTHUG.ZN[i, :, :2].T)
    ax[i].set_xlim([-5, 5])
    ax[i].set_ylim([-5, 5])

In [ ]:
compute_arctan = lambda point: math.atan2(*point[::-1])
rc('font',**{'family':'STIXGeneral'})

fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
for i in range(4):
    _ = ax[i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG.ZN[i, :, :2]), bins=30, density=True)
    ax[i].set_xticks([-math.pi, 0, math.pi])
    ax[i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i]))

In [ ]:
### This uses all N(T+1) particles and their w̄. 
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
for i in range(3):
    _ = ax[i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG.ZNK[i, :, :2]), bins=30, density=True, weights=MSTHUG.Wbar[i])
    ax[i].set_xticks([-math.pi, 0, math.pi])
    ax[i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i+1]))

# Markov Snippets THUG (version 2)

In [ ]:
# The aim now is to only output the end of the trajectory with ψ.
MSTHUG_ONLYEND = MarkovSnippetsTHUG(N=N, B=B, δ=δ, d=2, ϵs=ϵs, onlyend=True)

In [ ]:
zP_onlyend = MSTHUG_ONLYEND.sample()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
for i in range(4):
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value])
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value-MSTHUG_ONLYEND.ϵs[i], level_set_value+MSTHUG_ONLYEND.ϵs[i]], colors='gray')
    ax[i].scatter(*MSTHUG_ONLYEND.ZN[i, :, :2].T)
    ax[i].set_xlim([-5, 5])
    ax[i].set_ylim([-5, 5])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
for i in range(4):
    _ = ax[i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG_ONLYEND.ZN[i, :, :2]), bins=30, density=True)
    ax[i].set_xticks([-math.pi, 0, math.pi])
    ax[i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i]))

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
for i in range(3):
    _ = ax[i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG_ONLYEND.ZNK[i, :, :2]), bins=30, density=True, weights=MSTHUG_ONLYEND.Wbar[i])
    ax[i].set_xticks([-math.pi, 0, math.pi])
    ax[i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i+1]))

# SMC-THUG (or metropolised version)

In [ ]:
class MarkovSnippetsTHUGMetropolised:
    
    def __init__(self, N, B, δ, d, ϵs):
        """Metropolised version: for each particle compute the endpoint of trajectory and its weight.
        If the weight is positive, we accept the final point, otherwise we accept the initial point. 
        
        Parameters
        ----------
        
        :param N: Number of particles
        :type N:  int
        
        :param B: Number of bounces for the THUG integrator. Equivalent to `L` Leapfrog steps in HMC.
        :type B: int
        
        :param δ: Step-size used at each bounce, for the THUG integrator.
        :type δ: float
        
        :param d: Dimensionality of the `x` component of each particle, and equally dimensionality of 
                  `v` component of each particle. Therefore each particle has dimension `2d`.
        
        :param ϵs: Tolerances that fully specify the sequence of target filamentary distributions.
        :type ϵs: iterable
        """
        # Input variables
        self.N  = N       
        self.δ  = δ
        self.d  = d
        self.ϵs = ϵs       
        
        # Variables derived from the above
        self.P  = len(ϵs) - 1                                       # Number of target distributions
        self.ϖ  = MVN(zeros(d), eye(d))                             # Distribution of the velocities
        self.ηs = [generate_ηϵ(ϵ) for ϵ in ϵs]                      # List of filamentary distributions 
        
        self.B = B
    
    def initialize_particles(self):
        """To initialize particles, we sample from a uniform on a large rectangle.
        A rectangle of size [-100, 100] should be plenty large."""
        # Initialize particles by sampling from η_ϵ0 for a large ϵ0 which can be given as an argument.
        # Sample a point from the prior
        x0 = ellipse.sample(advanced=True) #np.random.uniform(low=-5, high=5, size=(self.d))
        # Use RWM starting from x0
        logηϵ0 = generate_logηϵ(self.ϵs[0])
        burn_in = 100
        TO_BE_THINNED, _ = RWM(x0, self.δ, burn_in + 10*self.N, logηϵ0)
        # Thin the samples to obtain the particles
        initialized_particles = TO_BE_THINNED[burn_in:][::10]
        v0 = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
        z0 = np.hstack((initialized_particles, v0))
        self.starting_particles = z0
        return z0
        
    def sample(self):
        """Starts the Markov Snippets sampler."""
        starting_time = time.time()
        # Initialize particles
        z = self.initialize_particles()   # (N, 2d)
        # Storage
        self.PARTICLES    = zeros((self.P+1, self.N, 2*self.d))
        self.PARTICLES[0] = z
        self.WEIGHTS      = zeros((self.P+1, self.N))
        self.WEIGHTS[0]   = 1 / self.N
        # For each target distribution, run the following loop
        for n in range(1, self.P+1):
            # Standard SMC sampler, we mutate the particles and then we resample
            ### Mutation step: 
            ###### Refresh velocities
            z[:, self.d:] = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
            ###### Mutate positions 
            M = lambda z: THUG_MH(z, B, self.δ, self.ηs[n-1])
            Z = np.apply_along_axis(M, 1, z)
            ### Compute weights
            w = (abs(np.apply_along_axis(f, 1, Z[:, :2]) - level_set_value) <= self.ϵs[n]).astype(float)
            w = w / w.sum()
            self.WEIGHTS[n] = w
            ### Resample
            indeces = choice(a=np.arange(self.N), size=self.N, p=w)
            z = z[indeces, :]
            self.PARTICLES[n] = z
        self.total_time = time.time() - starting_time
        return z

In [ ]:
MSTHUG_METROP = MarkovSnippetsTHUGMetropolised(N=N, B=B, δ=δ, d=2, ϵs=ϵs)
zP_metrop = MSTHUG_METROP.sample()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
for i in range(4):
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value])
    ax[i].contour(*prep_contour([-2.5, 2.5], [-2.5, 2.5], 0.01, f), levels=[level_set_value-MSTHUG_METROP.ϵs[i], level_set_value+MSTHUG_METROP.ϵs[i]], colors='gray')
    ax[i].scatter(*MSTHUG_METROP.PARTICLES[i, :, :2].T)
    ax[i].set_xlim([-5, 5])
    ax[i].set_ylim([-5, 5])

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
for i in range(3):
    _ = ax[i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG_METROP.PARTICLES[i, :, :2]), bins=30, density=True, weights=MSTHUG_METROP.WEIGHTS[i])
    ax[i].set_xticks([-math.pi, 0, math.pi])
    ax[i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i+1]))

### Compare times

In [ ]:
print("Markov Snippets Full Trajectory: {:.2}s".format(MSTHUG.total_time))
print("Markov Snippets End Only:        {:.2}s".format(MSTHUG_ONLYEND.total_time))
print("SMC-THUG (Metropolised version): {:.2}s".format(MSTHUG_METROP.total_time))

### Plot histograms together

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(12, 9), sharex=True, sharey=True)
# Markov Snippets - Full Trajectory (Version 1)
for i in range(3):
    _ = ax[0, i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG.ZNK[i, :, :2]), bins=30, density=True, weights=MSTHUG.Wbar[i])
    ax[0, i].set_xticks([-math.pi, 0, math.pi])
    ax[0, i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
    ax[0, i].set_title(r'$\mathregular{\epsilon}=$' + '{}'.format(ϵs[i+1]))
# Markov Snippets - End-Only (Version 2)
for i in range(3):
    _ = ax[1, i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG_ONLYEND.ZNK[i, :, :2]), bins=30, density=True, weights=MSTHUG_ONLYEND.Wbar[i])
    ax[1, i].set_xticks([-math.pi, 0, math.pi])
    ax[1, i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)
# SMC-THUG (Metropolised Version)
for i in range(3):
    _ = ax[2, i].hist(np.apply_along_axis(compute_arctan, 1, MSTHUG_METROP.PARTICLES[i, :, :2]), bins=30, density=True, weights=MSTHUG_METROP.WEIGHTS[i])
    ax[2, i].set_xticks([-math.pi, 0, math.pi])
    ax[2, i].set_xticklabels([r'$-\mathregular{\pi}$', r'$0$', r'$\mathregular{\pi}$'], fontsize=15)

In [ ]:
print("(Number of particles with non-zero weight for ϵ_P) / total sampling time: ")
print("MSv1: {:.1f}".format(np.sum(MSTHUG.Wbar[-1] > 0) / MSTHUG.total_time))
print("MSv2: {:.1f}".format(np.sum(MSTHUG_ONLYEND.Wbar[-1] > 0) / MSTHUG_ONLYEND.total_time))
print("SMC : {:.1f}".format(np.sum(MSTHUG_METROP.WEIGHTS[-1] > 0) / MSTHUG_METROP.total_time))

### Compute Means and other functionals

- use $\tilde{\psi} = \psi^T$ for $\tilde{T} = 1$
    - V1: keep initial and final point
    - V2: keep only final point
- Unfold ellipse $[0, 2\pi]$ and show histogram for both algorithms, use $N(T+1)$ particles for histogram. 
- ESS for $N(T+1)$ particles, and ESS for $2N$ particles (SMC version - count $1$s)
- Check functionals e.g. $x$, $x^2$

$$\mathbb{I}(\|f(x) - y\| \leq \epsilon)$$

$$
- \epsilon \leq f(x) - y \leq \epsilon
$$

$$
f^{(i)} 
$$